# Importing libraries

In [1]:
import pandas as pd
import io
import os

# Concatenating raw jsons

In [37]:
#reading raw json
raw_json_file_names = []
raw_jsons = []

for file in os.listdir("."):
    if file.startswith("run_results_"):
        raw_json_file_names.append(file)
        
for file in raw_json_file_names:
    with open(file,'r') as filestream:
        raw_jsons.append(pd.read_json(filestream))

In [38]:
#removing unnessasary stuff
for json_table in raw_jsons:
    del json_table['remove']
    del json_table['listingValue']
    json_table.columns = ['html']
    json_table['html_str'] = json_table['html'].astype(str)
    json_table = json_table.drop_duplicates(subset='html_str')

In [39]:
#concatenating jsons and removing duplicates
json_all = raw_jsons[0]
for json_table in raw_jsons[1:]:
    json_all = pd.concat([json_all,json_table], axis=0, ignore_index=True)
json_all = json_all.drop_duplicates(subset="html_str")
del json_all["html_str"]
json_all.reset_index(drop=True, inplace=True)

In [ ]:
#output raw json without duplicate as dataframe in pickle
with open('raw_json_no_dups.json','w') as fout:
    json_all.to_json(fout,orient='table')

# Getting only user messages
Also parsing links to message and authors

In [54]:
from bs4 import BeautifulSoup

In [55]:
#to add empty string in case of empty search
link_search = lambda s: '' if s is None else s['href']

In [56]:
#scrap user messages 
def scrapFromHTML(soup, row):
    row['author'] = "https://opendatascience.slack.com" + link_search(soup.find("a", class_="c-link c-message__sender_link"))
    row['link'] = link_search(soup.find("a", class_="c-link c-timestamp c-timestamp--static"))
    tag = soup.find("span", class_="c-message__body")
    if not tag is None:
        row['message'] = tag.get_text("\n")

In [57]:
#load raw table
with open('raw_json_no_dups.json','r') as fin:
    html_table = pd.read_json(fin,compression=None,orient='table')

In [58]:
#add tables for future scrapped data
html_table['author'] = ""
html_table['link'] = ""
html_table['message'] = ""

In [59]:
#scrapping of new information
for i in html_table.index:
    html_doc = html_table['html'][i]['name']
    soup = BeautifulSoup(html_doc,'html5lib')
    scrapFromHTML(soup, html_table.loc[i,:])

In [60]:
#copy table without non-user messages
html_table_usr_msg = html_table[html_table.author!="https://opendatascience.slack.com"]
html_table_usr_msg.reset_index(drop=True, inplace=True)
del html_table_usr_msg['html']

In [61]:
#output user messages table
with open("user_messages.json",'w') as fout:
    html_table_usr_msg.to_json(fout, compression=None, orient='table')

# Getting numbers out of text
And removing messages without numbers

In [2]:
import re

In [65]:
#input user messages table
with open("user_messages.json",'r') as fin:
    usr_msg = pd.read_json(fin, compression=None, orient='table')

In [66]:
usr_msg['numbers'] = usr_msg.apply(lambda x: [], axis=1)
usr_msg['hasNumbers'] = False

In [67]:
#scrapping numbers in messages
for i in usr_msg.index:
    msg = usr_msg['message'][i]
    usr_msg['numbers'][i] = re.findall(r'[0-9]+', msg)
    if usr_msg['numbers'][i]:
        usr_msg['hasNumbers'][i] = True 

/usr/lib/python3/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/lib/python3/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [68]:
#copy table without only-word messages (other are candidates for having vacancy information)
usr_msg_clear = usr_msg[usr_msg.hasNumbers]
usr_msg_clear.reset_index(drop=True, inplace=True)
del usr_msg_clear['hasNumbers']

In [69]:
#output clear user messages table
with open("user_messages_clear.json",'w') as fout:
    usr_msg_clear.to_json(fout, compression=None, orient='table')

# Parsing data and time

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
import time
from tqdm import tqdm
import threading

In [4]:
def scrapDateTime(row, link):

    usr = "ksetherg@gmail.com"
    pwd = "l'dbl1996"

    options = webdriver.chrome.options.Options()
    options.headless = True
    
    driver = webdriver.Chrome('/opt/chromedriver', options=options)
    driver.get(link.replace('archives','messages'))

    elem = driver.find_element_by_id("email")
    elem.send_keys(usr)
    elem = driver.find_element_by_id("password")
    elem.send_keys(pwd)
    elem.send_keys(Keys.RETURN)
#    elem.send_keys("Posted using Python's Selenium WebDriver bindings!")
#    elem = driver.find_element_by_id("signin_btn")
#    elem.click()
    try:
        WebDriverWait(driver,15).until(
             EC.presence_of_element_located((By.CLASS_NAME,'p-archives_banner')))
    except TimeoutException:
        pass
    
    try:
        WebDriverWait(driver,5).until(
            EC.presence_of_element_located((By.CLASS_NAME,'c-message_list__day_divider__label__pill')))
        elem1 = driver.find_element_by_class_name('c-message_list__day_divider__label__pill')
        row['date'] = elem1.text
    except TimeoutException:
        pass
    except StaleElementReferenceException:
        elem1 = driver.find_element_by_class_name('c-message_list__day_divider__label__pill')
        row['date'] = elem1.text
        
    try:
        WebDriverWait(driver,5).until(
            EC.presence_of_element_located((By.CLASS_NAME,'c-message__content_header')))
        elems = driver.find_elements_by_class_name('c-message__content_header')
        for elem in elems:
            if link in elem.get_attribute('innerHTML'):
                row['time'] = re.search('data-stringify-suffix="]"(.+?)</span>', elem.get_attribute('innerHTML')).group(1)
    except TimeoutException:
        pass
    except StaleElementReferenceException:
        elems = driver.find_elements_by_class_name('c-message__content_header')
        for elem in elems:
            if link in elem.get_attribute('innerHTML'):
                row['time'] = re.search('data-stringify-suffix="]"(.+?)</span>', elem.get_attribute('innerHTML')).group(1)
    finally:
        driver.quit()

In [5]:
#input user messages table
with open("user_messages_clear.json",'r') as fin:
    usr_msg_clear = pd.read_json(fin, compression=None, orient='table')

In [6]:
usr_msg_clear['date'] = ''
usr_msg_clear['time'] = ''

In [ ]:
for i in tqdm(usr_msg_clear.index[::10]):
    threads = []
    for j in range(i,min(i + 10,usr_msg_clear.index[-1])):
        thread = threading.Thread(target=scrapDateTime, args=[usr_msg_clear.loc[j,:], usr_msg_clear['link'][j]])
        threads.append(thread)
        thread.start()
    for thread in threads:
        thread.join()


  2%|▏         | 4/244 [17:23<16:28:40, 247.17s/it]Exception in thread Thread-58:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-4-2416fdc9f08c>", line 29, in scrapDateTime
    elem1 = driver.find_element_by_class_name('c-message_list__day_divider__label__pill')
  File "/home/rancher/.local/lib/python3.7/site-packages/selenium/webdriver/remote/webdriver.py", line 564, in find_element_by_class_name
    return self.find_element(by=By.CLASS_NAME, value=name)
  File "/home/rancher/.local/lib/python3.7/site-packages/selenium/webdriver/remote/webdriver.py", line 978, in find_element
    'value': value})['value']
  File "/home/rancher/.local/lib/python3.7/site-packages/selenium/webdriver/remote/webdriver.py", line 321, in execute
    self.error_handler.check_response(response)
  File "/home/r

In [ ]:
print(usr_msg_clear['date'])

In [ ]:
class SlackUser:

    def __init__(self, link='https://opendatascience.slack.com/archives/C04DA5FUF', filename='slack_dump.json'):
        self.data = {}
        self.channel_url = link
        self.filename = filename

        self._session: Optional[aiohttp.ClientSession] = None

    @property
    def session(self) -> aiohttp.ClientSession:
        if self._session is None:
            self._session = aiohttp.ClientSession()
        return self._session

    async def close(self):
        if self._session is not None:
            await self.session.close()

    def dump_exist(self, filename: str = None):
        return is_non_zero_file(filename or self.filename)

    def load(self, filename: str = None):
        with open(filename or self.filename, 'r', encoding='utf-8') as file:
            self.data = json.load(file)

    def dump(self, filename: str = None):
        with open(filename or self.filename, 'w', encoding='utf-8') as file:
            json.dump(self.data, file, indent=True, ensure_ascii=False)

    async def request(self, url, method='GET', data=None, headers=None):
        async with self.session.request(method, url, data=data, headers=headers) as response:
            return await response.read()

    async def authorization_on_slack(self, username, password):
        login_url = self.channel_url
        login_page = await self.request(login_url)
        auth_token = BeautifulSoup(login_page, features='lxml').find('input', {'name': 'crumb'})['value']

        data = {'_username': username, '_password': password, '_remember_me': 'on', 'crumb': auth_token}
        headers = {'Content-Type': 'application/x-www-form-urlencoded'}
        await self.request(f'{self.lmsu_url}/login_check', method='POST', data=data, headers=headers)

    async def scrap_user(self, user_id):
        response = await self.request(f'{self.lmsu_url}/rus/user/achievement/user/{user_id}/list')
        soup = BeautifulSoup(response, features="lxml")
        name_field = soup.find('h3', {'class': 'achievements-user__name'}) or \
        soup.find('a', {'class': 'user-block__link user-block__link--logged'})
        user_data = {'name': name_field.text.strip(), 'achievements': []}
        subsection(f'Processing user \"{user_data["name"]}\"')
        for achievement in soup.find_all("article", {"class": "achievement"}):
            curr_data = {
                          'title': achievement.find("a", {"class": "achievement__link"}).text.strip(),
                          'category': achievement.find("p", {"class": "achievement__more"}).text.strip(),
                          'score': int(achievement.find("span", {"class": "ach-pill"}).text),
                          'checked': bool(achievement.find("input", {"checked": "checked"})),
                          'url': self.lmsu_url + achievement.find("a", {"class": "achievement__link"})['href'],
                          'date': '',
                          'file': '',
                          'comment': '',
                          'comment_our': '',
                        }
            user_data['achievements'].append(curr_data)
        return user_id, user_data